In [1]:
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
train_df = pd.read_csv('/content/ratings_train.txt', encoding='utf8', sep='\t')

결측치 처리

In [5]:
train_df=train_df[train_df['document'].notnull()]

한글 정규화시키기

In [6]:
import re
train_df['document'] = train_df['document'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))

In [7]:
test_df=pd.read_csv('/content/ratings_test.txt',encoding='utf8',sep='\t')

In [8]:
test_df=test_df[test_df['document'].notnull()]

In [9]:
test_df['document'] = test_df['document'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))

한글을 토큰화 하기위한 라이브러리 설치

In [ ]:
#pip install konlpy

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
okt = Okt()

토큰화

In [12]:
tfidf = TfidfVectorizer(tokenizer= okt.morphs , ngram_range=(1, 2), min_df = 3, max_df = 0.9)

토큰화 시킨것 피트시키고 트렌스폼시키기

In [15]:
tfidf.fit(train_df['document'])
train_tfidf =tfidf.transform(train_df['document'])
train_tfidf

<149995x115612 sparse matrix of type '<class 'numpy.float64'>'
	with 2703859 stored elements in Compressed Sparse Row format>

In [16]:
tfidf.inverse_transform(train_tfidf)

[array(['짜증나네요', '진짜 짜증나네요', '진짜', '아', '목소리', '더빙 진짜', '더빙'], dtype='<U30'),
 array(['흠', '포스터 보고', '포스터', '초딩 영화', '초딩', '줄', '조차', '오버 연기', '오버',
        '영화 줄', '영화', '연기 조차', '연기', '않구나', '보고', '가볍지'], dtype='<U30'),
 array(['한 다', '한', '추천 한', '추천', '보는것을 추천', '보는것을', '무재', '래서', '다그', '다',
        '너 무재', '너'], dtype='<U30'),
 array(['평점 조정', '평점', '조정', '재미 는', '재미', '이야기', '없다 평점', '없다', '솔직히 재미',
        '솔직히', '는 없다', '는', '구먼', '교도소'], dtype='<U30'),
 array(['했던', '커스틴', '이뻐', '의', '영화', '연기 가', '연기', '에서', '스파이더맨', '사이',
        '보였다', '돋보였던 영화', '돋보였던', '늙어', '너무나도', '그 의', '그', '가 돋보였던',
        '가 너무나도', '가'], dtype='<U30'),
 array(['학년', '초등학교 학년', '초등학교', '움', '영화 ㅋㅋㅋ', '영화', '아까 움', '아까', '세',
        '생인', '부터', '별 반개', '별', '반개 도', '반개', '막', '마', '뗀', '도 아까', '도',
        '걸음', 'ㅋㅋㅋ 별', 'ㅋㅋㅋ'], dtype='<U30'),
 array(['했다', '제대로', '의 긴장감', '의', '을 제대로', '을', '원작 의', '원작', '긴장감 을',
        '긴장감'], dtype='<U30'),
 array(['해도 그것', '해도', '하는 사람', '하는', '정말', '인지 정말', '

In [17]:
test_tfidf = tfidf.transform(test_df['document'])

로지스틱회귀를 통해 정확도 측정

In [18]:
lr = LogisticRegression(random_state = 0)

In [19]:
lr.fit(train_tfidf, train_df['label'])

LogisticRegression(random_state=0)

In [20]:
test_predict = lr.predict(test_tfidf)

In [21]:
print('감성 분석 정확도 : ', round(accuracy_score(test_df['label'], test_predict), 3))

감성 분석 정확도 :  0.852


크롤링한 네이버 뉴스기사를 긍정 부정 판단 긍정 = 1 , 부정 = 0

In [60]:
news=pd.read_csv('/content/네이버뉴스_서울_폐업_20170101_20170131.csv',encoding='utf8')

In [61]:
news['title'] = news['title'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))
news_title=''
for i in range(len(news.title)):
  news_title=news_title + news.iloc[i,2]+''

print(news_title)

서울 중구 부동산중개업 등록할 때 면허세 고지서도 발급서울 영등포구 월일까지 공공일자리 참여구민 모집간호사들 서울로 서울로병동 폐쇄폐업까지르포 서울로 서울로지방병원 간호사 인력난라인뉴스 간호사들 서울로 서울로병동 폐쇄폐업까지꽁꽁 언 소비심리서울 주요 상권 위축서울시 청년여성 사업가 재기 지원매출 배서울시 특사경 무허가무면허 네일전문미용업소 곳 적발신년사박겸수 서울 강북구청장서울 약국가 경영난 때문에 개업보단 폐업서울경제 주택담보대출 연체해도 집 경매 년 유예해준다서울경제 금융위원회 올해 업무계획 주요 내용은년 안돼 폐업하는 약국 증가서울지역 부침 심해년 신년사 서울 강북구청장 박겸수신년사 박겸수 서울 강북구청장서울 핵심상권 빈 점포 급증 광교위례 신도시 상가도 짙은 불황년도금융동향서울 아파트 매매가 상승  서민금융확대  결혼 시 만원 공제종합박헌영 노승일 최순실로부터 포스트잇 받는 모습 목격거제통영고성 지난해 체불임금 년 비해  증가음식숙박업 생존기간 년 문제는 대출금리한국은 호랑이 멸종위기 부른 주요 국가였다줄잇는 폐업중고품도 안 팔린다경제교실 갈수록 느는 임금체불 해법 있나요식당모텔 곳 중 곳 창업 년 내에 문닫는다통행량 역대 최고길은 덜 막혀정치적 불확실성이 경기위축 유발자영업일용직 충격 커금리임대료 오르면 자영업 폐업률 급등대선 바로 알고 바로 뽑자대선주자 릴레이 인터뷰서울강원경기 개금리  오를땐 치킨집 폐업위험  자영업 폐업 위험도 보니금리 오르면 폐업률 급증   스티브 코언의 펀드업계 복귀 전략아무도안해서합니다 연말정산 간소화 서비스 정말 간편해졌나래빗 환불을 위한 헬스장은 없다대형마트 막아낸 망원시장 복합쇼핑몰 앞에 망연자실기획 자영업 점포의 생애 음식점 평균 년 도소매점 년에 비덱 수십억 송금 요구한 최순실거절당하자 까다롭게 군다충청브리핑 반기문 지지율 급감대권도전 중대기로일자리가 이다경제위기에도 노조는 기득권 사수 신규채용 꿈도투자자라면 꼭 봐야 할 오늘의 대 투자뉴스일공인중개사무소 만시대자격증 늘고 일감 줄고한은 음식점 평균 수명 년대출금리 

In [62]:
news_title_tfidf = tfidf.transform([news_title])
news_title_predict = lr.predict(news_title_tfidf)
news_title_predict

array([0])

In [63]:
news2=pd.read_csv('/content/네이버뉴스_서울_폐업_20170201_20170228.csv',encoding='utf8')

In [64]:
news2['title'] = news2['title'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))
news_title2=''
for i in range(len(news2.title)):
  news_title2=news_title2 + news2.iloc[i,2]+''

print(news_title2)

서울시 긴급복지 시스템으로 월 중 만 가구 지원작년 서울 불법대부업다단계 만건 적발서울시 민생침해모니터링단 명 활동 전개서울커피맵 커피공화국카페 점 찍으면 서울지도가 뜬다송파구 세모녀 잊지말자서울시 긴급복지 두달새 만가구서울 중구 주인 없이 방치된 노후 간판 무상 철거한경 미디어 뉴스룸한경닷컴 서울 커피점 년사커피빈 안정성 위 개서울 복판에 평짜리 복합문화공간이 꿈서울시 미니 태양광 지원 보조금 상향강화서울 동북구 유휴공간 자산화방안 머리 맞댄다서울시 지난해 태양광 미니발전소 만 가구 돌파반환점 돈 여성 첫 서울시의사회장 김숙희서울 중구 장기간 방치된 불량 간판 무상 철거 나서지난해 서울지역 약국 곳 문닫고 곳 신규 개업서울시 올해 미니태양광 만가구 보급서울의 호텔은 진화하고 있는가서울시 민생안전대책 성과공공일자리 개서울고법 현대기아차 포장출고업무 사내하청도 불법파견메르스 막다 망한 창원진앙지 삼성서울과 대조적포화된 서울 상권 성공 프랜차이즈를 위한 필수 소자본 창업아이템은현장르포 자영업자 한숨 중고물품 거래 서울 황학동 주방거리새주인 기다리는 중고주방용품사무장병원 조억 꿀꺽 환수율은 겨우 국립대 국유림 빌려 수목장 사업 가능해진다전국 주요 신문 톱뉴스일 조간안희정   장면으로 본 그의 삶 년 구멍가게 홀로서기 돕는 동작구공사한다고 하루 쉬라더니 자정에 문자로 해고신고 후 결격사유땐 갱신불허제 청담동 주식부자 막는다인터뷰  홍준표 경남도지사 대통령 되면 중국부터 가겠다단골술꾼들이 소설의 가장 훌륭한 인테리어였지요김정남 음력설때 교민에 조만간 술한잔하자 문자메시지창업자 블랙홀 치킨집월 매출 억의 조건은갈 곳 없는 푸드트럭약국 월 조제 부산 만원충남과 만원 격차경기불황 속 폐업속출쌓인 중고품 빌려입는 옷짙어지는 불황의 그늘내수 살리자금요일 조기퇴근 조기 예약하면 반값성공창업 상권을 보라패션 가두점의 현재와 미래조간 헤드라인 경기 일 화본 사람은 적은데 악플 너무 많아 이상하다 생각해영재센터 실소유주 누구최순실장시호 재판서 가린다성동구의사회 회원 가입 적극 독려사상

In [65]:
news_title_tfidf2 = tfidf.transform([news_title2])
news_title_predict2 = lr.predict(news_title_tfidf2)
news_title_predict2

array([0])

In [66]:
news3=pd.read_csv('/content/네이버뉴스_서울_폐업_20170301_20170331.csv',encoding='utf8')

In [67]:
news3['title'] = news3['title'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))
news_title3=''
for i in range(len(news3.title)):
  news_title3=news_title3 + news3.iloc[i,2]+''

print(news_title3)

서울시 치매노인 돌봄업체에 올해 억 투입서울경제 정유업계 호황인데 주유소는 폐업 속출공직자 재산등록박원순 서울시장 여전히 빚더미작년 서울 영세 휴대폰대리점 매출순이익  감소서울치킨맵 치킨 엑소더스 서막년 생존율 서울 명동서귀포 요우커 발길 뚝업계 월이 두렵다서울시 통신기기소매업 중소기업 적합업종 신청 지원민간기업 한국인삼공사 중소한약상 어렵게 해서울약령시그래픽뉴스 작년 서울 영세 휴대폰대리점 곳 중 곳 폐업 고려한국여행 금지령  유커 제주도 서울  감소할 듯서울 중구 지난해 위조상품 단속 최대 실적 실시간뉴스 헌재 앞 총집결 내일 서울에 최상위 경계령전문홍준표 보수우파 대통합 대통령되겠다 수락연설서울시 구의사회 정기총회 너무 썰렁했다자유한국당 대선후보로 홍준표 경남지사 선출 공감토론 근로시간 단축 논란 어떻게 풀 것인가우리 갑순이 송재림 묵언수행김소은 거짓임신 들통 위기홍준표 대통령 되면 기업 풀어주겠다제천소식한방바이오엑스포 공식여행사 공모 등마을을 먹여살리는 국가자본주의의 대안한국당 후보선출 모래시계 검사서 대선 후보로포토 세상주인잃은 주방기기만 가득채널 먹거리파일 대왕카스테라 관련 허위보도에 정정 요청공식홍준표 박근혜 그림자 딜레마가 숙제공약은 기업선택적 복지그의 시가 되어도 좋다 올어바웃 명태와 황태덕장개장 앞둔 제롯데월드 주변 상인들과 갈등은 여전자유한국당 대선후보 압승 거둔 홍준표자질 논란은 여전울산시장 조선업특별고용지원업종 지정연장 강력건의한국당 후보확정 홍준표 보수대통합 대통령될 것스트롱맨의 시대유커 없으니 제주 힐링 만끽 최 대리도 김 부장도 제주 가요자유한국당 대선후보 홍준표의 변신 자수성가형 검사홍트럼프한국당 후보선출홍준표 김영란법  규정 로 바꿀 것경제민 시민의 선택홍준표는 누구인가좌우 적폐 청산에 강한 의지 끝없김영란법 피해 가장 큰 업종은 일식당  매출 감소김기현 울산시장 조선업 특별고용 지원업종 지정 연장 건의근로시간 줄이자고 인력난에 밤샘근무 현실무시대왕카스테라 논란에 황교익 먹거리파일 선정적 방송 사과해야취업이 고민이라면 꿈같은 청

In [68]:
news_title_tfidf3 = tfidf.transform([news_title3])
news_title_predict3 = lr.predict(news_title_tfidf3)
news_title_predict3

array([0])

In [69]:
news4=pd.read_csv('/content/네이버뉴스_서울_폐업_20170401_20170430.csv',encoding='utf8')

In [70]:
news4['title'] = news4['title'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))
news_title4=''
for i in range(len(news4.title)):
  news_title4=news_title4 + news4.iloc[i,2]+''

print(news_title4)

주간 경매브리핑서울 아파트 낙찰가율  아래로 관망세서울 동대문구영등포구 행자부 지방규제개혁 평가 우수 지자체 선정서울시 자판기커피전문점 위생점검더치커피 등 선제적 실시한화큐셀 아시아 최대 재테크 박람회  서울머니쇼 참가서울 시내 자판기  위생 불량서울시 자판기  위생불량 코코아 세균수 초과서울경제투데이포커스가계대출 연체부담 완화책 보니서울시 커피전문점 증가 따라 밀착형 위생점검 실시서울시 유명 프로골퍼 아버지 등 고액체납자 가택수색창업부터 폐업까지 서울시 소상공인 밀착 지원규제개혁 상징 서울 호 푸드트럭 중고나라 매물 나온 이유는단독 대왕카스테라 서울 개업 올스톱줄폐업은 거짓서울시 창업부터 폐업까지 소상공인 밀착 지원창업부터 폐업까지 서울시 소상공인 밀착 지원홍준표 대통령 후보 서울 거점유세  자유대한민국 수호를 위한 서울 대첩 주서울역 고가공원 명암 서계동 봉제공장염천교 구둣방의 눈물한국행만 쏙 뺀 중국인 억천만명 노동절 연휴 여행너도 나도 사장님늘어난 일자리 둘 중 하나는 자영업남양주 규제개혁 역시 최고 대통령 표창인사이드 의사변호사 등 전문직 년 전망 밝을까박나래 측 최근 목 수술 받았다큰 병 아냐울분 찬 산부인과 의사들 소신진료 환경 보장하라난장이 아버지의 굴뚝이 대를 이어 울창하다박나래 나래바 임시폐업 선언수술 앞두고 술 정리하는 모습 애잔천여 영세 자영업자에 억 포털 광고 사기종합정부 호주산 신선란 운송비지원 월까지 연장박나래 입원 근황 공개 수술 나래바임시폐업남양주시 규제개혁 년 연속 대통령 표창박나래 나래바 임시폐업 이유는 목 수술성형수술 아님현장르포 사후면세점들 휴업상태 월 특수 기대 접었다망리단길 공방 사장님은 왜 절망했을까금호타이어 기업에 매각 안된다 지역민 반대 한목소리뉴스토리 프랜차이즈 가맹사업의 실태를 집중 조명한다폐업정보 플랫폼 리빌드 협력사 및 청년기업협회와 정재찬 가맹점사업자 관리업무 지자체에 위임 방안 마련스모크 연출 추정화 기대감 두렵지만 작품 갈증 원동력 공감토론 경제포커스 년간 연봉  올랐는데 세금  급증사무장병원 차리고 

In [71]:
news_title_tfidf4 = tfidf.transform([news_title4])
news_title_predict4 = lr.predict(news_title_tfidf4)
news_title_predict4

array([0])

In [72]:
news5=pd.read_csv('/content/네이버뉴스_서울_폐업_20170501_20170531.csv',encoding='utf8')

In [73]:
news5['title'] = news5['title'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))
news_title5=''
for i in range(len(news5.title)):
  news_title5=news_title5 + news5.iloc[i,2]+''

print(news_title5)

서울경제투데이포커스 법정 최고금리 인하 부작용 없나서울 푸드트럭 대로 늘린다야시장도 곳으로 확대서울 강남 유명 웨딩홀 폐업예비부부 쌍 날벼락킹키부츠 년 월 서울 공연 확정오디션 개최 공식서울시 푸드트럭 영업활성화 강력 모색영업지 곳 푸드트럭 대 추가 운포토 폐업 위기 겪고 일어선 시네마달 블랙리스트 어워드 수상 서울환서울시 글로벌 부동산중개사무소 개로 확대 운영서울시 외국어 서비스 부동산 인증 업소 모집서울중앙지법 가습기 살균제 제조업체 배상 책임 있다서울환경영화제 블랙리스트 어워드 시상한다서울중앙지법 가습기 살균제 제조업체 배상 책임 있다대한통운 소상공인 성장 지원한다택배 할인 실시서울시 글로벌 부동산중개사무소 확대 지정 운영대접받던 우량 임차인 은행웨딩홀 이젠 찬밥신세검찰 폐업한 부곡하와이 경영비리 의혹 수사재계 최저임금 인상 여부에 초긴장경제 위축 고려해야아산 성매매 우려지역 걷기 투어 효과 톡톡서울환경영화제 변호인과 시네마달 대표에 블랙리스트 어워드 시상국민카드 소상공인 위한 골목상권 민관 공동연구 착수진주의료원에서 강제로 옮겨진 환자 시간만에 사망일 한국 레코드 역사를 다시 쓴다헤지펀드 거물 스티브 코언 역대 최대펀드로 재기 노려이것은 현대예술이다맥주 파는 심야 갤러리와 극장 투어형 공연부가세 원천징수 추진에 카드업계 유탄억 부정 수급 사무장병원 적발정기예금라던 선물옵션 쪽박분통 터진 대 투자자부곡하와이 년 만에 폐업추억 간직하겠습니다보건산업노조 강제폐업된 진주의료원 재개원해야로열티를 싫어하는 프랜차이즈 기업들소득분포 공개 선진국보다 큰 계층간 소득임금 격차 줄여야통합 정책 펼칠 컨트롤타워 기대정기예금라던 선물옵션 쪽박분통 터진 대 투자자부동산토리 조원 뉴딜 강북을 주목하라부곡하와이 마지막 영업년 서민 휴양지 문 닫는다최윤 아프로서비스그룹 회장 종합금융사로 나아간다컬투쇼 오연서 부곡하와이 폐업 오랜 추억 잃는 듯소득분배 악화 속도외환위기 후 최고더벨크레디아그리콜 증권 대표 새로 뽑는다라이프 사이트 가입코인 충전으로운동 언제 어디서나 오늘은 이런 경향월

In [74]:
news_title_tfidf5 = tfidf.transform([news_title5])
news_title_predict5 = lr.predict(news_title_tfidf5)
news_title_predict5

array([0])

In [75]:
news6=pd.read_csv('/content/네이버뉴스_서울_폐업_20170601_20170630.csv',encoding='utf8')

In [76]:
news6['title'] = news6['title'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))
news_title6=''
for i in range(len(news6.title)):
  news_title6=news_title6 + news6.iloc[i,2]+''

print(news_title6)

서울경제 최저임금 만원 인상 논란 쟁점향방은서울 헬스장 소비자 피해 속출예방 요령은서울 기숙사지하상가숙박시설 곳 화재대비 불량서울시의사회 제증명 수수료 규제 불필요서울시 금감원시중은행과 금융사기 예방 맞손서울 대부업체 위법행위 년 새 배 증가서울시 서울창업허브 중심으로 키친 인큐베이팅 나선다금감원 서울시금융권과 민생침해 피해예방  체결서울시금감원시중은행 금융사기 예방 나선다올봄 서울지역 화재 건 줄고 피해액 억원 늘고서울 중구약사회 중구보건소와 약사 현안 논의서울시금감원은행 상조예치금 확인 시스템 만든다년만에 서울에 생긴 공장서울시내 활동 만화가 명 중 명 인권침해 경험 있어서울 실업자 명 중 명 대년제 대학 학력자 서울시 양천구 시니어스 케어 통합재가장기요양기관 개소서울시 생활권계획 덮어놓고 상권 확대하다 문 닫는 가게만 늘어날까 우려이기화 한국여성경제인협회 서울지회장 고객이 안 오시면 직접 찾아가야죠한계상황에 내몰린 서울시내 지하상가 년  비해 매출 반토막서울로  개장 길 하나 두고 만리  서계 희비서울 실업자 그들은 룸펜 청송합니다끝까지 추적해 징수한다 까지 쫓아온 서울시고용부 호식이두마리치킨 억만원 임금 미지급 시정 지시최저임금 개 업종 차등화 꺼낸 경영계노사 셈법 더 복잡해져서울광장 문재인표 환경정책의 출발선진경호 부국장 겸 사회부장정부 부자증세 공식화중산서민층 세제지원은 확대종합 공감토론 최저임금 만원 인상 논란 쟁점은정부 내년 지방선거 이후 조세개혁 단계적 추진소상공인 어려움 외면마라 국회에 추경처리 촉구김삼화 조대엽 리서치 사임서 효력 위법행위 면피용휴후맛있는 휴가하 물회어죽재첩국가는 곳마다 군침창간특집새롭게 등장한 사회서비스공단장기요양 대변화 불러오나정부 부자증세 공식화 조세개혁 특위 설치집권 년차 증세 폭풍부자증세 추진하되 영세업자엔 세제 편의 제공납세환경 급변상속세도 유산취득세로 전환해야조대엽 고용노동부 장관 후보자 인사청문회 올해  수산 직불금 고등어민대구 등 개 품목국정위 하반기 조세개혁 특별위 설치 세제 개편 논의국정위 하반기 조세재정

In [77]:
news_title_tfidf6 = tfidf.transform([news_title6])
news_title_predict6 = lr.predict(news_title_tfidf6)
news_title_predict6

array([0])

In [78]:
news7=pd.read_csv('/content/네이버뉴스_서울_폐업_20170701_20170731.csv',encoding='utf8')

In [79]:
news7['title'] = news7['title'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))
news_title7=''
for i in range(len(news7.title)):
  news_title7=news_title7 + news7.iloc[i,2]+''

print(news_title7)

개장 년 서울시 자영업지원센터전문가 명이 컨설팅폐업 속출 푸드트럭서울은 대만 상시 운영서울시 금천직장맘지원센터 개소 년 직장 내 고충 상담개장 년 맞은 서울시 자영업지원센터서울시의회 박중화 의원 전기생활용품 안전관리법 개정 건의안 대표발의내일부터 서울세종 등 청약조정지역  강화서울금융복지상담센터 주년 성과는서울시 자영업지원센터 주년 소상공인 만여명 도움받아최저임금 주기 어렵다서울시내 상가 매물 쏟아져서울시치과의사회 이벤트 치과 상술 넘어가면 안돼 당부서울경제 중기 수출 첨병 를 만나다 미래코리아서울경제 최저임금 원 경제실험 성공할까젠트리피케이션 방지책 가시화되나서울 성동구 상생협약 확대 서울미래유산 그랜드 투어 경성의 월스트리트 한국은행 앞 광장오늘부터 서울 등 부동산 대출 규제 강화서울 등  오늘부터 강화 대출 규모 줄어요창원고성진주순천광주서울일간 국토종단 도주극양도세  늘어난 조강남구가 서울 절반 차지서울경제 임대주택 공급 문닫은 병원주유소 활용해야창원 진주 순천 서울 일간 도주극이데일리 서울세종 등 청약조정지역  강화 폐업 폐업때 점포 철거는 서울시 지원프로 통해 최소비용으로 가능서울세종 등 청약조정지역  강화된다파워 인터뷰 이근재 외식업 서울시협의회장 최저임금 인상 인건비 부담 탈원전 신에너지시대 박원순 서울시의 원전 하나줄이기태양광 지원 확오늘부터 서울경기 등 청약조정지역  강화된다폐업 알리는 인제군 식당뿔난  수첩은 민노총 청부 제작소인가 성토 서울미래유산 그랜드투어 청기와 얹은 국기원 시대극 단골 허바홍기영칼럼 위기의 자영업자 구하기맛집이 뭐길래커피왕은 왜 죽음을 선택했나탈세 혐의 김정규 타이어뱅크 회장 구속영장 기각종합단독 성공창업부터 사기혐의 몰락까지 스베누 사태 풀스토리바른정당 가맹점 갑질 근절 위한 정책간담회 개최탈세 혐의 김정규 타이어뱅크 회장 구속영장 실질심사사드 폭격 맞은 동대문메르스때보다 심하다탈세 혐의 김정규 타이어뱅크 회장 구속영장 청구치킨의 레드오션 깨뜨리고 문화외식 융합공간으로 신선한 즐거움을 선사하는서민 금융부담 줄이려면 최

In [80]:
news_title_tfidf7 = tfidf.transform([news_title7])
news_title_predict7 = lr.predict(news_title_tfidf7)
news_title_predict7

array([0])

In [81]:
news8=pd.read_csv('/content/네이버뉴스_서울_폐업_20170801_20170831.csv',encoding='utf8')

In [82]:
news8['title'] = news8['title'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))
news_title8=''
for i in range(len(news8.title)):
  news_title8=news_title8 + news8.iloc[i,2]+''

print(news_title8)

단독 서울시 역점사업 행복나무 마을활력소 년 만에 사실상 문 닫아상가임대차분쟁 소송없이서울시에 요청하세요서울 부동산중개업소 만천여개강남구 천개 위서울창업허브 키친 인큐베이팅 프로그램 진행서울창업허브 키친 인큐베이팅 그랜드 오픈서울형 도시재생 용산전자상가 일대 창업의 메카로 부활 날개 짓서울 중개업소 만곳 영업중강남구 압도적 위서울시 푸드트럭 영업공간 추가확보 수익성 개선될까서울시 푸드트럭 상시영업 가능한 곳 추가 확대서울시 성동구 젠트리피케이션 임대료 안정세 보여서울에서 부동산 중개업소 가장 많은 지역은백 투 더 서울 탄 서울 예술가 따라잡기서울경제 통신비 인하 통신사 왜 나만 갖고 그래서울양양고속도로 개통 한 달동해안 성황내륙은 비상박에 만원 농어촌 민박이 호화펜션 둔갑취재파일 대통령의 수제화 다시 만날 날이 온다없어서 못 팔아요 강남 부자들 뭉칫돈 채권 파생에 몰린다음식점창업 뭣이 중헌디 유명세보다 맛으로 승부해야 롱런국민 위한 문재인 케어 의료계는 성났다생계가 걸려 있다 레진 웹소설 작가진 갑작스러운 서비스 종료에 분노박스조합 골판지 대기업 횡포 막아달라공정위에 진정서 제출전국 주요 신문 톱뉴스일 조간도봉구 방학천 유해업소 없애고 한글문화거리로정부 비급여항목 전면급여화 정책 철회하라팬택 특허처분 인력 제외 노동자 대다수 해고브리핑이재용 부회장 심에서 징역 년글로벌 포커스 중국의 북한 급변사태 대응 플랜내년 예산안이런 사업도 있다이색사업 선가습기살균제 폐손상 단계 피해 심사계획 의결 연기차 구제계정운트렌드 전쟁에서 패한 제너럴 밀스르포재창업 선택 아닌 필수 실패 경험은 자산기억해야 할 역사 원양어업 년 원양산업 발전정책 토론회권영찬 특허권은 반드시 보호돼야 된다대책 투기과열지구 지정 서면으로 급행 심사종합스타트업 대동여지도 울산광역시단독 액세서리 프랜차이즈 레드아이 폐업 수순가맹점주들 보증금 반환 이색예산사업뱀장어 전용 고속도로  개발유커 공백 올해 관광수지 적자 달러 예상연대보증제 내년 폐지하고 법정 최고금리 로 낮춰대한민국 청년 신화 빛과 그림자전국의 

In [83]:
news_title_tfidf8 = tfidf.transform([news_title8])
news_title_predict8 = lr.predict(news_title_tfidf8)
news_title_predict8

array([0])

In [84]:
news9=pd.read_csv('/content/네이버뉴스_서울_폐업_20170901_20170930.csv',encoding='utf8')

In [85]:
news9['title'] = news9['title'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))
news_title9=''
for i in range(len(news9.title)):
  news_title9=news_title9 + news9.iloc[i,2]+''

print(news_title9)

대책이 기회 서울 부동산중개업소 되레 늘었다 대책에도 서울 부동산 중개업소 늘어나명도소송에서 패소한 서울미래유산 공씨책방대책에도 서울 부동산 중개업소 증가살 떡집살 다방서울 오래가게로 오게서울영등포역 등 민자역사 곳 년 임시사용허가서울 중구 도로 위 주인없는 장기 방치 간판 감사원 서울시농수산식품공사 노조부당지원수사요청감사원 서울시공사 위탁수수료 계약집행 부당처리서울 은평구 정신건강증진센터 횡령고용승계 문제로 마찰서울시 프랜차이즈 가맹 계약 전 무료 상담 받으세요양범진 떡보의 하루 인천서울경기지역 본부장 갑질은 남 얘기죠전철수 서울시의원 소상공인 지원 특위 위원장에 선출자치광장 소셜벤처에서 발견한 새로운 미래정원오 서울 성동구청장브리핑도시바  마무리 마침내 웃은 면세점 업계 풍요로운 한가위 볼멘 소리 왜조선소 물량팀 노동자들 황금연휴는 딴 나라 이야기사드 보복에  전담여행사 절반 이상 휴폐업황찬중 시의원 내년 춘천시장 출마 공식 선언연휴에 뭐할까 미술관서 만나는 한국사회의 민 낯은문구업계 다이소 탓 경영난다이소 납득어렵다 반박   정몽준 현대중공업그룹 최대주주편의점 식품위생법 위반건수 년새 배 적발 위연예인도 치료 유명 치과환자 천여 명 돈 먹튀면세점의 눈물  본격화 된 사업 철수 도미노 인천공항도 철수 나올까 공감토론 파리바게트 직접 고용 지시와 히딩크 감독 영입 논란국제 핫이슈중 대북제재 대폭 강화석유제품 수출 제한북한 기업 폐더 못 버틴다 줄줄이 문 닫는 면세점천상 맨 유인촌 전 장관 어쩜 그리 뻔뻔하신가전국 곳 현장노동청 다 들어 줍니다단독 연예인 치료 환자 모은 치과 먹튀피해자 천 명증축공사 고대안암병원문전약국 지형도 변화 예고폐업하고 이름 바꾸면 끝두번세번 속는 온라인몰 먹튀폐쇄 연기로 한숨 돌린 언양시외버스터미널 추석연휴 정상운영돈 되는 이색 투자 종 눈길실업률 역주행  년 연속한국은 년 연속다이소 때문에 문구점  매출 떨어져부정청탁금지법 년 잠재적 범죄자 시선에 위축당장 죽게생겼는데기재부 제도개선안에 면세업계 한숨아쉽지만주간의 효리네 민박 웃으며

In [86]:
news_title_tfidf9 = tfidf.transform([news_title9])
news_title_predict9 = lr.predict(news_title_tfidf9)
news_title_predict9

array([0])

In [87]:
news10=pd.read_csv('/content/네이버뉴스_서울_폐업_20171001_20171031.csv',encoding='utf8')

In [88]:
news10['title'] = news10['title'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))
news_title10=''
for i in range(len(news10.title)):
  news_title10=news_title10 + news10.iloc[i,2]+''

print(news_title10)

서울은 강북 골목상권 전성시대강남대형 상권 대신 떠오른 강북 꼬마 상현장의 시각언양터미널 폐업 서울주 발전의 전화위복 계기로비즈 르포 예비창업자 위한 꿈의 공간 서울창업허브 프라이빗 키친 가노인장기요양 인정률서울  전북 격차 케어 서울시에 중앙시장 마지막 개도살 판매업소 폐쇄 조치 촉구서울시 폐업 상담 전담창구 개설서울시 폐업 고민 소상공인 돕는다서울시 국감 야당  박원순 쟁점 포인트 셋서울시 전국 최초 공공보건의료재단 출범 제의 진주의료원 사태 막는 공공서울시 서울신보에 폐업 지원 전담창구 개설강남 고급 베이비스튜디오 제니스 스튜디오 서울베이비페어 참가자치광장 도시재생 문화가 답이다이동진 서울 도봉구청장모닝오늘 서울시 국감 결국 박원순 때리기  올레드  번인 없다더국감년새 서울 푸드트럭 대 폐업행정절차 특수성 때문년 국정감사서울중부지방국세청 납세자보호담당관 신설 등 납세자서울시 폐업 고민하는 소상공인을 위한 전담창구 개설서울로 서울로 충북의료 불신 심각대책에도 서울 부동산 중개업소 증가전월 대비 개소 늘어인권위도 혁신위 출범 보수정권서 잃어버린 년 되찾나언양시외버스터미널 역사 속으로 사라져정부 대책에도 서울 부동산 중개업소는 오히려 늘어지난해 국내 기업 원 벌면 원 남겼다평화문화진지 시작에서 완공까지파이낸셜포커스법정 최고금리  내년 월 시행 저축서 밀려난 저신용자뉴스체크경제 상조업체 소비자 피해 주의경기도 소상공인 사업정리도우미 성과마크롱 개혁 약발 통했나붙는 경제공공조직은행 비리 문제 심각 인체조직 채취 도축에 비교홍종학 논란 옹호하는 국감선 부 대물림사립교 비난종합 국감자질논란 확산 홍종학 청문회 험로 예고종합대  가게 나오셔야할판알바 줄이거나 문닫을수도유산균 등 건기식 부작용 년간 천건 넘게 발생국회 통과한 복지관련 개정법의 주요내용여행 믿고 갔는데관광공사 인증 부실 숙박시설인 제조 단계의 제언 단계  단계다주택 대출 더 죈다 만기 제한 도입함영주 국감에서 하나은행의 박근혜 게이트 연루 추궁받아미혼모 낙인에 해고폐업도피 중고 겪었죠문재인 구두 아지오 폐

In [89]:
news_title_tfidf10 = tfidf.transform([news_title10])
news_title_predict10 = lr.predict(news_title_tfidf10)
news_title_predict10

array([0])

In [90]:
news11=pd.read_csv('/content/네이버뉴스_서울_폐업_20171101_20171130.csv',encoding='utf8')

In [91]:
news11['title'] = news11['title'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))
news_title11=''
for i in range(len(news11.title)):
  news_title11=news_title11 + news11.iloc[i,2]+''

print(news_title11)

관악구 서울시 지적토지업무 평가 최우수구하이서울브랜드기업 포커스 해외에서 막막할 때 영사관보다 빠른 배달의 서울시 스타트업 육성은 립서비스주목 우리 동네 상권핫 스팟핫 플레이스 서울의 브루클린으로 뜨는 성유커 돌아온다 서울 관광업계 기대감백 투 더 서울  탄끝위기에 처한 서울미래유산밀려나는 유산들한미 차 공청회 일 개최농축산업계도 참여하루 아침에 호텔 폐업 노동자들 생존권 위협금리인상 긴축시대 예고 임금 올랐는데 금리도 애타는 중기소상뉴스 늘어나는 이자만 조원 항체 될까 폭탄 될까한국을 움직이는 국회의원  김성식 국민의당 의원서울 관악구갑   하이서울브랜드 혁신기업 열전  건강 바이오로 퍼플오션 시장을 개척신해철 집도의 근황 전남 모병원 외과 과장으로 재직너의 등짝에 스매싱 윤서현 자유분방한 스타일의 욜로족 의사 변신의 지나친 시장개입 국토부 신규 심사 늑장 이유는부산 창간 주년 중견기업 늘고 영업이익률 전국 톱  부산경제 탄탄해졌사은품 붙여팔기추가금 요구 상조업체 소비자 피해주의보아베의 경제 살리기 사천리관악 부동산 원스톱 서비스지적토지업무 최우수구로디휴브 중고가전 매입 중개서비스 피커스 리뉴얼수제화 공임보다 비싼 커피 카페거리 젊은층 반갑지만은 않다트렌드를 읽는 책 위기의 자영업자그 많던 사장님들 다 어디로 갔을까 서경 베스트히트상품 효원상조아산시 장미마을 유흥주점 건물철거 본격화긴축의 시대 돈파티 끝났다중기소상공인 최저임금 이어 금리까지생트빅투아르 산에서 세잔에게 좋은 그림 물어봤다너의 등짝에 스매싱 파이팅폭력 남편 빚 떠안은 아내폐지 모으는 할머니장기소액연체자 면면 보니판결단독 고속도로 건설 소음으로 양돈농가 폐업했다면실낱같은 희망도 무산농축산화훼농가 반발한지완 가려도 예쁨 너의 등짝에 스매싱 제작발표회정보요원 모집중대검 범정 기능 부활 움직임 포착하이킥순풍 뭉쳤다박해미박영규 스매싱 시트콤 어벤저스의 역습돈 안 주면 성매매 허위 신고협박에 우는 시각장애 안마사들고가 전자제품 내걸고상조업계 사은품 꼼수 주의보공인중개사무소 거래 절벽경쟁 격화에 폐업 속출맛

In [92]:
news_title_tfidf11 = tfidf.transform([news_title11])
news_title_predict11 = lr.predict(news_title_tfidf11)
news_title_predict11

array([1])

In [94]:
news12=pd.read_csv('/content/네이버뉴스_서울_폐업_20171201_20171231.csv',encoding='utf8')

In [95]:
news12['title'] = news12['title'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))
news_title12=''
for i in range(len(news12.title)):
  news_title12=news_title12 + news12.iloc[i,2]+''

print(news_title12)

회사서 도망쳤어요서울 실업자  취업 경험자고통받는 사람들도 함께서울 도심 소외계층 성탄미사종로 뜨고 강남 지고희비 엇갈린 서울 상권서울 노란우산공제 가입자 년새 송파구  서울시 민원서비스개선 우수사례 선정데이터 정책제안 서울시 데이터 콘트롤 타워가 없다서울 민원서비스 개선 최우수사례 시민 평가로 뽑는다서울시 내년 월일까지 소방안전교육 받으세요단독 빵의 눈물올해만 서울 빵집 곳 무더기 폐업형간염 집단발병 서울현대의원서 양성확인 명명 치료중서울 송파구 불법 현수막간판 대대적 정비 나서송파구  서울시 민원서비스개선 우수사례 선정서울시의회 사회적경제 성장 토론회 개최즈위슬랏의 한국 블로그서울의 체인점 커피 너무 비슷한 맛서울시 다중이용업소 관계자소방안전교육 이수 해야서울시민이 뽑은 최고 민원서비스는자신감 낚고 다시 서울로아내에게 감사김근태상 윤민석 내 노래는 마음의 상처에 붙이는 대일밴드 역할김동환의 일요세상 쓰레기장도 아닌데 담배꽁초가 왜 이곳에혹시 당신이인간으로 최소한 살 수 있게는 해줘야죠년 식어가는 태양 앞에 난장이들이 매달려 있다최저임금김상조청탁금지법전안법소상공인 대 뉴스배틀그라운드 장병규 돈 없어 창업 못해 반 맞고 반 틀려키워드로 본  대한민국 한국호 시대성공 열쇠는 년 중소기업계 이슈 최저임금 인상근로시간 단축 등 내년까지 이어질가장 바쁜 연말 대리운전기사들이 촛불을 든 이유국민건강보험공단 이사장에 김용익 전 의원 임명 일 본회의 제안이견 현안과 민생법안 분리하자르포년만에 첫삽상암롯데몰 호재에 전화 폭주인근 뉴타운도 들썩푸드트럭 타사광고 허용광고수익 창출 기대소상공인연합회 전안법 이슈 등 소상공인 대 뉴스 발표공인중개업소 경기 빨간불 커졌지만 개업은 계속 늘어나대전 리베라호텔 추억 기억하시나요표류하는 전안법옷장신구 등 인증비 폭탄만 소상공인 범법자로 내몰뺑소니 사고 운전자에 최대 만원 부담금 굴뚝에 산타말고 노동자가 있다법정최고금리 인하년 바뀌는 금융제도 가지하도급거래 공정화김상조의 세 번째 칼날 불공정 하도급 관행 이진구 기자의 최저임금 결정에 문 대

In [96]:
news_title_tfidf12 = tfidf.transform([news_title12])
news_title_predict12 = lr.predict(news_title_tfidf12)
news_title_predict12

array([1])